In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [20]:
# 브라우저 실행 -> 네이버지도 이동
driver = wb.Chrome()
driver.get("https://map.naver.com/p?c=15.00,0,0,0,dh")
driver.maximize_window()

In [21]:
# 검색창에 동명동맛집 입력 -> 검색
# 선택자가 랜덤으로 변하는 경우가 있다.
# How? 변하지 않는 선택자를 찾자 ex) class, 계층선택자
search = driver.find_element(By.CSS_SELECTOR,"input.input_search")
search.send_keys("동명동맛집")
search.send_keys(Keys.ENTER)

In [7]:
# 검색된 가게들의 태그를 수집
# Why-> 하나씩 클릭하기 위해서
title = driver.find_elements(By.CSS_SELECTOR,"#_pcmap_list_scroll_container > ul > li > div.CHC5F > a.tzwk0")
len(title)

0

## 화면에는 보이는데.. 접근이 안된다..?
- iframe을 의심해봐라
- iframe = 다른 서버에 저장된 데이터를 보여주기만 하는 태그
- 우리가 필요한건 실제 서버의 주소 -> src속 경로를 파악했다.
- 상대경로이면 -> 절대경로로 변경 -> req로 크롤링할 때 사용하는 방식
- But! selenium은 브라우저 자체를 제억하는 목적 -> 링크를 옮겨가는게 아닌 화면에서 조작 실행
- How? driver가 현재 가지고있는 화면을(처음 검색된 결과) iframe으로 변경

In [22]:
# 원본데이터에서 iframe으로 변경
# 이동할 iframe태그 찾기 -> 드라이버에게 변경 명령 내리기
# 원본에서 iframe으로 변경 -> switch_to.frame(iframe의 태그위치(선택자))
search_iframe = driver.find_element(By.CSS_SELECTOR,"#searchIframe")
driver.switch_to.frame(search_iframe)

In [9]:
# 다시 수집해보기
title = driver.find_elements(By.CSS_SELECTOR,"#_pcmap_list_scroll_container > ul > li > div.CHC5F > a.tzwk0")
len(title)

30

In [23]:
# 전체 결과가 아니다 -> 스크롤링 필요
# 기존방식 : body태그에게 키보드의 end를 보내는 방식을 사용
# iframe인 경우에는 body에게 스크롤을 명령할 수가 없다.
# 해결책 : iframe속 데이터를 담고있는 태그(div) -> 옆에 scroll이라고 태그가 붙은 div가 존재
div = driver.find_element(By.CSS_SELECTOR,"#_pcmap_list_scroll_container")
div.send_keys(Keys.END)

In [24]:
# 기존스크롤 : 임의 횟수만큼 for으로 작성
# 단점 : 반복 횟수가 낭비 될 수 있다, 부족할 수 도있다.
# while문을 통해서 알아서 끝가지 반복하는 문법
# 대화 : 스크롤을 내리기전 데이터와 스크롤을 내린 후 데이터를 비교해라
# 만약에 두 데이터가 다르다면 ? 스크롤이 된다 -> 스크롤 해라
# 만약에 두 데이터가 같다면? 스크롤이 안된다 -> 스크롤을 멈춰라
div = driver.find_element(By.CSS_SELECTOR,"#_pcmap_list_scroll_container")
while True :
    # 내리기 전 데이터를 저장
    first_data = driver.page_source
    # 스크롤을 내리기
    div.send_keys(Keys.END)
    # 시간부여
    time.sleep(0.5)
    # 내린 후 데이터를 저장
    last_data = driver.page_source
    # 멈추는 조건 부여
    if first_data == last_data :
        break

In [ ]:
# 다른 코드에 활용할 때는 div -> body태그로 변경
body = driver.find_element(By.CSS_SELECTOR,"body")
while True :
    first_data = driver.page_source
    body.send_keys(Keys.END)
    time.sleep(0.5)
    last_data = driver.page_source
    if first_data == last_data :
        break

In [25]:
# 스크롤후 데이터 다시 수집
title = driver.find_elements(By.CSS_SELECTOR,"#_pcmap_list_scroll_container > ul > li > div.CHC5F > a.tzwk0")
len(title)

76

In [26]:
# 첫번째 가게 클릭 -> 디테일 정보
title[0].click()

In [ ]:
# 디테일 창에서 가게이름 수집
# 새로운 iframe으로 변환
entry_iframe = driver.find_element(By.CSS_SELECTOR,"#entryIframe")
driver.switch_to.frame(entry_iframe)

In [27]:
# 해당 코드가 오류가 나는 이유
# 2번 iframe과 3번 iframe은 서로 모르는 관계
# how? 2번 -> 원본 -> 3번 
# iframe에서 원본으로 이동하는 방법
driver.switch_to.default_content()

In [28]:
# 다시 iframe으로 이동
entry_iframe = driver.find_element(By.CSS_SELECTOR,"#entryIframe")
driver.switch_to.frame(entry_iframe)

In [29]:
# 디테일 속의 가게 이름 수집
store_name = driver.find_element(By.CSS_SELECTOR,"#_title > div > span.GHAhO")
store_name.text

'쭈꾸미잡스'

In [30]:
# 목적 : 2번째 검색 결과를 클릭 후, 디테일 정보 중 가게타이틀 수집
# 키포인트 : 현재 위치는 3번 iframe
# 1) 원본으로 이동 2) 2번 iframe으로 이동 3) 가게클릭 4) 원본이동 5) 3번 iframe이동 6) 수집
driver.switch_to.default_content()
driver.switch_to.frame(search_iframe)
title[1].click()
time.sleep(1)
driver.switch_to.default_content()
entry_iframe = driver.find_element(By.CSS_SELECTOR,"#entryIframe")
driver.switch_to.frame(entry_iframe)
store_name = driver.find_element(By.CSS_SELECTOR,"#_title > div > span.GHAhO")
store_name.text

'아이다요'

In [33]:
# 반복문을 통해서 모든 가게 정보 수집
name_list = []
for i in range(len(title)) :
    driver.switch_to.default_content()
    driver.switch_to.frame(search_iframe)
    title[i].click()
    time.sleep(3)
    driver.switch_to.default_content()
    entry_iframe = driver.find_element(By.CSS_SELECTOR,"#entryIframe")
    driver.switch_to.frame(entry_iframe)
    store_name = driver.find_element(By.CSS_SELECTOR,"#_title > div > span.GHAhO")
    name_list.append(store_name.text)

KeyboardInterrupt: 

# 오늘 파일 정리
- 아이디가 바뀌는 선택자 존재 -> class, 계층을 활용하자
- selenium에서 iframe을 처리하는 방법
- 1) 원본데이터 -> iframe 이동 = driver.swtich_to.frame(이동할 iframe 위치)
  2) frame -> frame 불가능 = frame -> 원본 -> frame 이동
     - driver.switch_to.default_content()로 이동 후에 변환 가능
- 태그를 수집할 때 그냥 검색된거 수집하지 말고 태그를 분석하자
  - 글자를 클릭했을 때 화면이 바뀐다면 반드시 a태그를 수집
  - 글자를 입력하는 공간을 수집할 때는 반드시 input태그를 수집하자
  - 광고같은 키워드로 인해서 클릭이 되지 않는 경우 -> 큰 영역을 수집하자
- iframe에서 스크롤 작업할 때 body가 안되는 경우가 있다
  - 영역을 포함한 div들 중 옆에 태그로 scroll을 확인후, 그 div에게 end 보내기
- 앞으로 스크롤 내리는 작업할 때는 while문 코드 사용하기

### 최종코드 통합본

In [ ]:
#브라우저 실행
driver = wb.Chrome()
driver.get("https://map.naver.com/p?c=15.00,0,0,0,dh")
driver.maximize_window()
time.sleep(1)
# 검색어 입력
search = driver.find_element(By.CSS_SELECTOR,"input.input_search")
search.send_keys("동명동맛집")
search.send_keys(Keys.ENTER)
time.sleep(1)
# 검색결과 iframe 이동
search_iframe = driver.find_element(By.CSS_SELECTOR,"#searchIframe")
driver.switch_to.frame(search_iframe)
# 스크롤
div = driver.find_element(By.CSS_SELECTOR,"#_pcmap_list_scroll_container")
while True :
    first_data = driver.page_source
    div.send_keys(Keys.END)
    time.sleep(0.5)
    last_data = driver.page_source
    if first_data == last_data :
        break
# 가게리스트 수집 -> 클릭하기 위해서
title = driver.find_elements(By.CSS_SELECTOR,"#_pcmap_list_scroll_container > ul > li > div.CHC5F > a.tzwk0")
# 반복해서 수집
name_list = []
for i in range(len(title)) :
    driver.switch_to.default_content()
    driver.switch_to.frame(search_iframe)
    title[i].click()
    time.sleep(3)
    driver.switch_to.default_content()
    entry_iframe = driver.find_element(By.CSS_SELECTOR,"#entryIframe")
    driver.switch_to.frame(entry_iframe)
    store_name = driver.find_element(By.CSS_SELECTOR,"#_title > div > span.GHAhO")
    name_list.append(store_name.text)